In [2]:
import sympy as sp

sp.init_printing(use_latex=True)

L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x_B, y_B, z_B, yaw, pitch, roll, q1, q2, q3, alpha, beta, z_CS = sp.symbols('x_B y_B z_B yaw pitch roll q_1 q_2 q_3 alpha beta z_CS', real=True)
state_vector = sp.Matrix([x_B, y_B, z_B, roll, pitch, yaw, q1, q2, q3])
joint_vector = sp.Matrix([q1, q2, q3])

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

# Elementary rotation matrices
def Rx(x):
    return sp.Matrix([[1, 0, 0], [0, sp.cos(x), -sp.sin(x)], [0, sp.sin(x), sp.cos(x)]])

def Ry(y):
    return sp.Matrix([[sp.cos(y), 0, sp.sin(y)], [0, 1, 0], [-sp.sin(y), 0, sp.cos(y)]])

def Rz(z):
    return sp.Matrix([[sp.cos(z), -sp.sin(z), 0], [sp.sin(z), sp.cos(z), 0], [0, 0, 1]])

### Frame definitions
The nominal configuration (i.e. all joint angles 0) is defined as the arm pointing upwards and being fully extended. The frames for this derivation are defined as follows:
- **Inertial frame** $\mathcal{F}_{I}$ - Frame attached to the world, unmoving, North-East-Down for positive XYZ axes.
- **Body frame** $\mathcal{F}_{B}$ - Frame attached to the drone body in its center of mass (CoM), moves and rotates with body, Forward-Right-Down for positive XYZ axes
- **Manipulator base frame** $\mathcal{F}_{0}$ - Origin coincides with body frame, but rotated to Up-Right-Forward for positive XYZ to align Z axis with rotation of $q_{1}$ and set Y axis such that Euler angle singularity in $\mathcal{F}_{0}$ is placed away from operational workspace. Does not rotate with $q_{1}$
- **Frame 1** $\mathcal{F}_{1}$ - Z-axis coincides with axis of rotation of $q_{2}$. X points towards $q_{3}$ when $q_{2}=0$. Does not rotate with $q_{2}$
- **Frame 2** $\mathcal{F}_{2}$ - Z-axis coincides with axis of rotation of $q_{3}$. X points towards end-effector. Does not rotate with $q_{3}$
- **Frame S** $\mathcal{F}_{S}$ - Oriented with positive Z pointing inwards, X in direction of negative $Z_{3}$ and Y completing right handed
- **Frame C** $\mathcal{F}_{C}$ - Oriented perpendicular to the sensor, contact frame.

<p align="center">
    <img title="Kinematic diagram" src="images/kinematic_diagramv2(1).png">
</p>

### Angle conventions
The transformation between $\mathcal{F}_{S}$ and $\mathcal{F}_{S}$ uses the extrinsic XYZ Euler angle convention with angles $\alpha$, $\beta$, and $\gamma$ denoting the rotations about XYZ, respectively.


In [3]:
# Inertial to Body
R_B = Rz(yaw)*Ry(pitch)*Rx(roll)
P_B = sp.Matrix([[R_B[0,0], R_B[0,1], R_B[0,2], x_B],
                 [R_B[1,0], R_B[1,1], R_B[1,2], y_B],
                 [R_B[2,0], R_B[2,1], R_B[2,2], z_B],
                 [0, 0, 0, 1]])
sp.pprint(str(R_B).replace('sin', 'np.sin').replace('cos', 'np.cos'))

# Body to manipulator base frame (frame 0)
P_B0 = sp.Matrix([[0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [0, 0, 0, 1]])

# Manipulator base frame (frame 0) to frame 1
R_01 = Rz(q1)*Rx(-sp.pi/2)
P_01 = sp.Matrix([[sp.cos(q1), 0, -sp.sin(q1), sp.cos(q1)*L1],
                  [sp.sin(q1), 0, sp.cos(q1), sp.sin(q1)*L1],
                  [0, -1, 0, 0],
                  [0, 0, 0, 1]])

# Frame 1 to frame 2
R_12 = Rz(q2)*Rx(sp.pi/2)
P_12 = sp.Matrix([[sp.cos(q2), 0, sp.sin(q2), sp.cos(q2)*L2],
                  [sp.sin(q2), 0, -sp.cos(q2), sp.sin(q2)*L2],
                  [0, 1, 0, 0],
                  [0, 0, 0, 1]])

# Frame 2 to Sensor frame
R_2S = Rz(q3)*Ry(-sp.pi/2)
P_2S = sp.Matrix([[0, -sp.sin(q3), -sp.cos(q3), sp.cos(q3)*L3],
                  [0, sp.cos(q3), -sp.sin(q3), sp.sin(q3)*L3],
                  [1, 0, 0, 0],
                  [0, 0, 0, 1]])

# Sensor frame to contact frame
# 1. Contact frame to sensor frame
R_CS = Ry(beta)*Rx(alpha)
P_CS = sp.Matrix([[R_CS[0,0], R_CS[0,1], R_CS[0,2], 0],
                  [R_CS[1,0], R_CS[1,1], R_CS[1,2], 0],
                  [R_CS[2,0], R_CS[2,1], R_CS[2,2], z_CS],
                  [0, 0, 0, 1]])

# 2. Sensor frame to contact frame
R_SC = R_CS.T
p_CS = sp.Matrix([[0], [0], [z_CS]])

P_SC = sp.Matrix([[R_SC[0,0], R_SC[0,1], R_SC[0,2], (-R_SC@p_CS)[0]],
                  [R_SC[1,0], R_SC[1,1], R_SC[1,2], (-R_SC@p_CS)[1]],
                  [R_SC[2,0], R_SC[2,1], R_SC[2,2], (-R_SC@p_CS)[2]],
                  [0, 0, 0, 1]])
sp.pprint(P_SC)

Matrix([[np.cos(pitch)*np.cos(yaw), np.sin(pitch)*np.sin(roll)*np.cos(yaw) - n
p.sin(yaw)*np.cos(roll), np.sin(pitch)*np.cos(roll)*np.cos(yaw) + np.sin(roll)
*np.sin(yaw)], [np.sin(yaw)*np.cos(pitch), np.sin(pitch)*np.sin(roll)*np.sin(y
aw) + np.cos(roll)*np.cos(yaw), np.sin(pitch)*np.sin(yaw)*np.cos(roll) - np.si
n(roll)*np.cos(yaw)], [-np.sin(pitch), np.sin(roll)*np.cos(pitch), np.cos(pitc
h)*np.cos(roll)]])
⎡   cos(β)         0        -sin(β)         z_CS⋅sin(β)    ⎤
⎢                                                          ⎥
⎢sin(α)⋅sin(β)  cos(α)   sin(α)⋅cos(β)  -z_CS⋅sin(α)⋅cos(β)⎥
⎢                                                          ⎥
⎢sin(β)⋅cos(α)  -sin(α)  cos(α)⋅cos(β)  -z_CS⋅cos(α)⋅cos(β)⎥
⎢                                                          ⎥
⎣      0           0           0                 1         ⎦


In [4]:
# Printing the equations
print("----- ACTUAL POSE OF C IN S -----")
for i in range(P_SC.shape[0]):
    for j in range(P_SC.shape[1]):
        print(f"self.P_SC[{i},{j}] = {str(P_SC[i,j]).replace("cos", "np.cos").replace("sin", "np.sin")}")
print("\n ----- REFERENCE POSE OF S IN C -----")
for i in range(P_CS.shape[0]):
    for j in range(P_CS.shape[1]):
        print(f"self.P_CS[{i},{j}] = {str(P_CS[i,j]).replace("cos", "np.cos").replace("sin", "np.sin")}")

----- ACTUAL POSE OF C IN S -----
self.P_SC[0,0] = np.cos(beta)
self.P_SC[0,1] = 0
self.P_SC[0,2] = -np.sin(beta)
self.P_SC[0,3] = z_CS*np.sin(beta)
self.P_SC[1,0] = np.sin(alpha)*np.sin(beta)
self.P_SC[1,1] = np.cos(alpha)
self.P_SC[1,2] = np.sin(alpha)*np.cos(beta)
self.P_SC[1,3] = -z_CS*np.sin(alpha)*np.cos(beta)
self.P_SC[2,0] = np.sin(beta)*np.cos(alpha)
self.P_SC[2,1] = -np.sin(alpha)
self.P_SC[2,2] = np.cos(alpha)*np.cos(beta)
self.P_SC[2,3] = -z_CS*np.cos(alpha)*np.cos(beta)
self.P_SC[3,0] = 0
self.P_SC[3,1] = 0
self.P_SC[3,2] = 0
self.P_SC[3,3] = 1

 ----- REFERENCE POSE OF S IN C -----
self.P_CS[0,0] = np.cos(beta)
self.P_CS[0,1] = np.sin(alpha)*np.sin(beta)
self.P_CS[0,2] = np.sin(beta)*np.cos(alpha)
self.P_CS[0,3] = 0
self.P_CS[1,0] = 0
self.P_CS[1,1] = np.cos(alpha)
self.P_CS[1,2] = -np.sin(alpha)
self.P_CS[1,3] = 0
self.P_CS[2,0] = -np.sin(beta)
self.P_CS[2,1] = np.sin(alpha)*np.cos(beta)
self.P_CS[2,2] = np.cos(alpha)*np.cos(beta)
self.P_CS[2,3] = z_CS
self.P_CS[3,0] = 0

In [5]:
# Sensor pose in inertial frame
P_S = sp.trigsimp(P_B*P_B0*P_01*P_12*P_2S)
sp.pprint(P_S)

⎡-(sin(pitch)⋅cos(yaw)⋅cos(q₁ + roll) + sin(yaw)⋅sin(q₁ + roll))⋅sin(q₂) + cos
⎢                                                                             
⎢(-sin(pitch)⋅sin(yaw)⋅cos(q₁ + roll) + sin(q₁ + roll)⋅cos(yaw))⋅sin(q₂) + sin
⎢                                                                             
⎢                       -sin(pitch)⋅cos(q₂) - sin(q₂)⋅cos(pitch)⋅cos(q₁ + roll
⎢                                                                             
⎣                                                  0                          

(pitch)⋅cos(q₂)⋅cos(yaw)  -(-(sin(pitch)⋅cos(yaw)⋅cos(q₁ + roll) + sin(yaw)⋅si
                                                                              
(yaw)⋅cos(pitch)⋅cos(q₂)  -((-sin(pitch)⋅sin(yaw)⋅cos(q₁ + roll) + sin(q₁ + ro
                                                                              
)                                                                   -(sin(pitc
                                                   

In [6]:
# Printing the equations
print("----- ACTUAL POSE OF S IN I -----")
for i in range(P_S.shape[0]):
    for j in range(P_S.shape[1]):
        print(f"P_S[{i},{j}] = {str(P_S[i,j]).replace("cos", "np.cos").replace("sin", "np.sin")}")

----- ACTUAL POSE OF S IN I -----
P_S[0,0] = -(np.sin(pitch)*np.cos(yaw)*np.cos(q_1 + roll) + np.sin(yaw)*np.sin(q_1 + roll))*np.sin(q_2) + np.cos(pitch)*np.cos(q_2)*np.cos(yaw)
P_S[0,1] = -(-(np.sin(pitch)*np.cos(yaw)*np.cos(q_1 + roll) + np.sin(yaw)*np.sin(q_1 + roll))*np.cos(q_2) - np.sin(q_2)*np.cos(pitch)*np.cos(yaw))*np.sin(q_3) + (np.sin(pitch)*np.sin(q_1 + roll)*np.cos(yaw) - np.sin(yaw)*np.cos(q_1 + roll))*np.cos(q_3)
P_S[0,2] = -(-(np.sin(pitch)*np.cos(yaw)*np.cos(q_1 + roll) + np.sin(yaw)*np.sin(q_1 + roll))*np.cos(q_2) - np.sin(q_2)*np.cos(pitch)*np.cos(yaw))*np.cos(q_3) - (np.sin(pitch)*np.sin(q_1 + roll)*np.cos(yaw) - np.sin(yaw)*np.cos(q_1 + roll))*np.sin(q_3)
P_S[0,3] = -L_1*np.sin(pitch)*np.cos(yaw)*np.cos(q_1 + roll) - L_1*np.sin(yaw)*np.sin(q_1 + roll) - L_2*np.sin(pitch)*np.cos(q_2)*np.cos(yaw)*np.cos(q_1 + roll) - L_2*np.sin(q_2)*np.cos(pitch)*np.cos(yaw) - L_2*np.sin(yaw)*np.sin(q_1 + roll)*np.cos(q_2) + L_3*np.sin(pitch)*np.sin(q_3)*np.sin(q_1 + roll)*np.cos(yaw)

In [7]:
P_BS = P_B0*P_01*P_12*P_2S

# Printing the equations
print("----- ACTUAL POSE OF S IN B -----")
for i in range(P_BS.shape[0]):
    for j in range(P_BS.shape[1]):
        print(f"P_S[{i},{j}] = {str(P_BS[i,j]).replace("cos", "np.cos").replace("sin", "np.sin")}")

----- ACTUAL POSE OF S IN B -----
P_S[0,0] = np.cos(q_2)
P_S[0,1] = np.sin(q_2)*np.sin(q_3)
P_S[0,2] = np.sin(q_2)*np.cos(q_3)
P_S[0,3] = -L_2*np.sin(q_2) - L_3*np.sin(q_2)*np.cos(q_3)
P_S[1,0] = np.sin(q_1)*np.sin(q_2)
P_S[1,1] = -np.sin(q_1)*np.sin(q_3)*np.cos(q_2) + np.cos(q_1)*np.cos(q_3)
P_S[1,2] = -np.sin(q_1)*np.cos(q_2)*np.cos(q_3) - np.sin(q_3)*np.cos(q_1)
P_S[1,3] = L_1*np.sin(q_1) + L_2*np.sin(q_1)*np.cos(q_2) + L_3*np.sin(q_1)*np.cos(q_2)*np.cos(q_3) + L_3*np.sin(q_3)*np.cos(q_1)
P_S[2,0] = -np.sin(q_2)*np.cos(q_1)
P_S[2,1] = np.sin(q_1)*np.cos(q_3) + np.sin(q_3)*np.cos(q_1)*np.cos(q_2)
P_S[2,2] = -np.sin(q_1)*np.sin(q_3) + np.cos(q_1)*np.cos(q_2)*np.cos(q_3)
P_S[2,3] = -L_1*np.cos(q_1) - L_2*np.cos(q_1)*np.cos(q_2) + L_3*np.sin(q_1)*np.sin(q_3) - L_3*np.cos(q_1)*np.cos(q_2)*np.cos(q_3)
P_S[3,0] = 0
P_S[3,1] = 0
P_S[3,2] = 0
P_S[3,3] = 1


In [ ]:
# Testing inverse kinematics
P_S = P_S.subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])
# List the state dependencies of all elements:
# for i in range(P_S.shape[0]):
#     for j in range(P_S.shape[1]):
#         sp.pprint(f"P_S[{i},{j}] = f({P_S[i,j].free_symbols})")

# Case 1: Zero state -> Identity rotation and -(L1+L2+L3) in Z
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, 0), (roll, 0), (q1, 0), (q2, 0), (q3, 0)])
assert(P_S_subbed == sp.Matrix([[1, 0, 0, 0],[0, 1, 0, 0],[0, 0, 1, -(L1_length+L2_length+L3_length)],[0, 0, 0, 1]]))

# Case 2: pi/4 on q3.
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, 0), (roll, 0), (q1, 0), (q2, 0), (q3, sp.pi/4)])
assert(P_S_subbed == sp.Matrix([[1, 0, 0, 0],[0, sp.cos(sp.pi/4), -sp.sin(sp.pi/4), sp.sin(sp.pi/4)*L3_length],[0, sp.sin(sp.pi/4), sp.cos(sp.pi/4), -(L1_length+L2_length+sp.cos(sp.pi/4)*L3_length)],[0, 0, 0, 1]]))

# Case 3: pi/4 on q2
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, 0), (roll, 0), (q1, 0), (q2, sp.pi/4), (q3, 0)])
assert(P_S_subbed == sp.Matrix([[sp.cos(sp.pi/4), 0, sp.sin(sp.pi/4), -sp.sin(sp.pi/4)*(L2_length + L3_length)],[0, 1, 0, 0],[-sp.sin(sp.pi/4), 0, sp.cos(sp.pi/4), -(L1_length+sp.cos(sp.pi/4)*(L3_length+L2_length))],[0, 0, 0, 1]]))

# Case 4: pi/4 on q1
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, 0), (roll, 0), (q1, sp.pi/4), (q2, 0), (q3, 0)])
assert(P_S_subbed == sp.Matrix([[1, 0, 0, 0],[0, sp.cos(sp.pi/4), -sp.sin(sp.pi/4), sp.sin(sp.pi/4)*(L1_length+L2_length+L3_length)],[0, sp.sin(sp.pi/4), sp.cos(sp.pi/4), -sp.cos(sp.pi/4)*(L1_length+L3_length+L2_length)],[0, 0, 0, 1]]))

# Case 5: pi/6 on pitch
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, sp.pi/6), (roll, 0), (q1, 0), (q2, 0), (q3, 0)])
assert(P_S_subbed == sp.Matrix([[sp.cos(sp.pi/6), 0, sp.sin(sp.pi/6), -(L1_length+L2_length+L3_length)*sp.sin(sp.pi/6)],[0, 1, 0, 0],[-sp.sin(sp.pi/6), 0, sp.cos(sp.pi/6), -sp.cos(sp.pi/6)*(L1_length+L3_length+L2_length)],[0, 0, 0, 1]]))

# Case 6: pi/6 on roll
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, 0), (roll, sp.pi/6), (q1, 0), (q2, 0), (q3, 0)])
assert(P_S_subbed == sp.Matrix([[1, 0, 0, 0],[0, sp.cos(sp.pi/6), -sp.sin(sp.pi/6), sp.sin(sp.pi/6)*(L1_length+L2_length+L3_length)],[0, sp.sin(sp.pi/6), sp.cos(sp.pi/6), -sp.cos(sp.pi/6)*(L1_length+L3_length+L2_length)],[0, 0, 0, 1]]))

# Case 7: pi/3 on q1 and np.pi/6 on q3
P_S_subbed = P_S.subs([(x_B, 0), (y_B, 0), (z_B, 0), (yaw, 0), (pitch, 0), (roll, 0), (q1, sp.pi/3), (q2, 0), (q3, sp.pi/6)])
assert(P_S_subbed == sp.Matrix([[1, 0, 0, 0],[0, 0, -1, sp.sin(sp.pi/3)*(L1_length + L2_length)+L3_length],[0, 1, 0, -(L1_length + L2_length)*sp.cos(sp.pi/3)-sp.cos(sp.pi/3+sp.pi/6)*L3_length],[0, 0, 0, 1]]))


⎡1  0  0           0        ⎤
⎢                           ⎥
⎢0  0  -1  0.273 + 0.2105⋅√3⎥
⎢                           ⎥
⎢0  1  0        -0.2105     ⎥
⎢                           ⎥
⎣0  0  0           1        ⎦
⎡1  0  0           0        ⎤
⎢                           ⎥
⎢0  0  -1  0.273 + 0.2105⋅√3⎥
⎢                           ⎥
⎢0  1  0        -0.2105     ⎥
⎢                           ⎥
⎣0  0  0           1        ⎦


In [47]:
P_C = P_S @ P_SC
P_C = sp.simplify(P_C)
# Printing the equations
print("----- ACTUAL POSE OF C IN I -----")
for i in range(P_C.shape[0]):
    for j in range(P_C.shape[1]):
        print(f"P_C[{i},{j}] = {str(P_C[i,j]).replace("cos", "np.cos").replace("sin", "np.sin")}")

----- ACTUAL POSE OF C IN I -----
P_C[0,0] = np.sin(beta)*np.sin(pitch)*np.sin(alpha - q_3)*np.sin(q_1 + roll)*np.cos(yaw) + np.sin(beta)*np.sin(pitch)*np.cos(q_2)*np.cos(yaw)*np.cos(alpha - q_3)*np.cos(q_1 + roll) + np.sin(beta)*np.sin(q_2)*np.cos(pitch)*np.cos(yaw)*np.cos(alpha - q_3) - np.sin(beta)*np.sin(yaw)*np.sin(alpha - q_3)*np.cos(q_1 + roll) + np.sin(beta)*np.sin(yaw)*np.sin(q_1 + roll)*np.cos(q_2)*np.cos(alpha - q_3) - np.sin(pitch)*np.sin(q_2)*np.cos(beta)*np.cos(yaw)*np.cos(q_1 + roll) - np.sin(q_2)*np.sin(yaw)*np.sin(q_1 + roll)*np.cos(beta) + np.cos(beta)*np.cos(pitch)*np.cos(q_2)*np.cos(yaw)
P_C[0,1] = -np.sin(pitch)*np.sin(alpha - q_3)*np.cos(q_2)*np.cos(yaw)*np.cos(q_1 + roll) + np.sin(pitch)*np.sin(q_1 + roll)*np.cos(yaw)*np.cos(alpha - q_3) - np.sin(q_2)*np.sin(alpha - q_3)*np.cos(pitch)*np.cos(yaw) - np.sin(yaw)*np.sin(alpha - q_3)*np.sin(q_1 + roll)*np.cos(q_2) - np.sin(yaw)*np.cos(alpha - q_3)*np.cos(q_1 + roll)
P_C[0,2] = np.sin(beta)*np.sin(pitch)*np.sin(q_2)*n

In [1]:
import numpy as np
state = np.array([1, 3, 4, 5, 6])
weighting_matrix = np.eye(5)
weighting_matrix[4,4] = 100
weighting_matrix[2,2] = 0.001
res = np.matmul(state, np.matmul(weighting_matrix, state))
print(res)

3635.016
